In [121]:
import pandas as pd
import statsmodels.api as sm
from IPython.display import Markdown
from scipy import stats
from sklearn.model_selection import train_test_split

from src.report import (
    t_test,
    f_test,
    general,
    general_assessment,
    dw_test,
    bg_test
)
from src.stats import LinearRegression, add_constant

In [122]:
%reload_ext autoreload
%autoreload 2

In [123]:
ALPHA = 0.05

In [124]:
DATA = """
1 50 120
2 58 115
3 60 116
4 54 130
5 56 132
6 50 130
7 68 118
8 70 118
9 55 128
10 66 115
11 80 112
12 85 105
13 95 128
14 96 135
15 108 132
16 185 205
"""

DATA = [map(int, row.split()[1:]) for row in DATA.strip().split('\n')]

In [125]:
DATA = pd.DataFrame(DATA, columns=['y', 'x'])
DATA

,y,x
0,50,120
1,58,115
2,60,116
3,54,130
4,56,132
5,50,130
6,68,118
7,70,118
8,55,128
9,66,115


In [126]:
X = add_constant(pd.concat([
    DATA['x'].rename('x1'),
    DATA['x'].shift(1).rename('x2'),
    DATA['x'].shift(2).rename('x3')
], axis=1))
y = DATA['y']
X

,const,x1,x2,x3
0,1.0,120,NaN,NaN
1,1.0,115,120.0,NaN
2,1.0,116,115.0,120.0
3,1.0,130,116.0,115.0
4,1.0,132,130.0,116.0
5,1.0,130,132.0,130.0
6,1.0,118,130.0,132.0
7,1.0,118,118.0,130.0
8,1.0,128,118.0,118.0
9,1.0,115,128.0,118.0


In [127]:
X_train, X_test, y_train, y_test = train_test_split(X.dropna(), y.iloc[2:], train_size=0.98, shuffle=False)
X_train

,const,x1,x2,x3
2,1.0,116,115.0,120.0
3,1.0,130,116.0,115.0
4,1.0,132,130.0,116.0
5,1.0,130,132.0,130.0
6,1.0,118,130.0,132.0
7,1.0,118,118.0,130.0
8,1.0,128,118.0,118.0
9,1.0,115,128.0,118.0
10,1.0,112,115.0,128.0
11,1.0,105,112.0,115.0


In [128]:
model = LinearRegression(y_train, X_train)

In [129]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                 -0.285
Method:                 Least Squares   F-statistic:                    0.1122
Date:                Sat, 18 Dec 2021   Prob (F-statistic):              0.951
Time:                        16:31:15   Log-Likelihood:                -55.738
No. Observations:                  13   AIC:                             119.5
Df Residuals:                       9   BIC:                             121.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        130.9894    139.354      0.940      0.3

D:\IT\Coding\GitHub Projects\My-University\III Курс\(python) Эконометрика\venv\Lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [130]:
general(model)

$ y_i = a + b_{1} x_{i1} + b_{2} x_{i2} + b_{3} x_{i3} + e_i $<br>
<br>
$ \hat{y_i} = \hat{a} + \hat{b_{1}} x_{i1} + \hat{b_{2}} x_{i2} + \hat{b_{3}} x_{i3} $<br>
<br>
$ \hat{y_i} = 130.989 + -0.078 x_{i1} + 0.058 x_{i2} + -0.464 x_{i3} $<br>
<br>
$ S_\hat{a} = 139.354 $<br>
$ S_{\hat{b_{1}}} = 0.799 $<br>
$ S_{\hat{b_{2}}} = 0.829 $<br>
$ S_{\hat{b_{3}}} = 0.872 $

In [131]:
general_assessment(model)

### Оценка среднеквадратического отклонения возмущений.<br>
<br>
$ \sqrt{\frac{1}{n - k - 1} \sum_{i=1}^{n}{(y_i - \hat{y_i}})^2} $<br>
<br>
$ S_{e} = 21.167 $<br>
$ 30 \% < 2116.71_{S_{ei}} \% \rightarrow $ оценка: <b>0/10</b>.<br>
<br>
<br>
### Коэффициент детерминации.<br>
<br>
$ R^2 = 1 - \frac{\sum_{i=1}^{n}{(y_i - \hat{y_i})^2}}{\sum_{i=1}^{n}{(y_i - \bar{y})^2}}  = \frac{\sum_{i=1}^{n}{(\hat{y_i} - \bar{y})^2}}{\sum_{i=1}^{n}{(y_i - \bar{y})^2}}  = 1 - \frac{RSS}{TSS} = \frac{ESS}{TSS} $<br>
<br>
$ R^2 = 0.036 $<br>
$ 0.5 > 0.036 \rightarrow $ оценка: <b>0/10</b>.<br>
<br>
<br>
### Средняя относительная ошибка аппроксимации.<br>
<br>
$ A = \frac{1}{n} \sum_{i=1}^{n}{|\frac{y_i - \hat{y_i}}{y_i}|} \cdot 100 \% $<br>
<br>
$ A = 21.163 $<br>
$ 15 \% < 21.163 \rightarrow $ оценка: <b>0/10</b>.

In [132]:
f_test(model)

### F-критерий Фишера.
<br><br>
$ H_0: b_1 = ... = b_k = 0, \\ H_1: b_1^2 + ... + b_k^2 > 0. $
<br><br>
$$ F = \frac{\frac{R^2}{k}}{\frac{1 - R^2}{(n - k - 1)}} $$
<br><br>
$ F_{набл} = 0.112 $<br>
$ p\text{-}value_F = 0.951 $
<br><br>
$ \alpha = 0.05 $<br>
$ F_{табл_{0.05}} = 3.048 $
<br><br>
$ 0.951 > 0.05 $<br>
$ p\text{-}value > \alpha \rightarrow $ гипотеза $ H_0 $ <b>принимается</b> - модель в целом <b>незначима</b>.

In [133]:
t_test(model)

### t-критерий Стьюдета для оценки значимости параметров модели.
<br><br>
$ H_0: b_i = 0, \\ H_1: b_i \neq 0. $
<br><br>
$$ t_{b_i} = \frac{\hat{b_i}}{S_{\hat{b_i}}} $$
<br><br>
$ \alpha = 0.05 $
<br>
$ t_{табл_{0.05}} = 2.262 $
<br>

<br>
$ t_{a} = 0.940 $, $ p\text{-}value_{t_{a}} = 0.372 $, $ p\text{-}value > \alpha \rightarrow $гипотеза $ H_0 $ <b>принимается</b> - параметр <b>незначим</b>.<br>$ t_{b_{1}} = -0.098 $, $ p\text{-}value_{t_{b_{1}}} = 0.924 $, $ p\text{-}value > \alpha \rightarrow $гипотеза $ H_0 $ <b>принимается</b> - параметр <b>незначим</b>.<br>$ t_{b_{2}} = 0.070 $, $ p\text{-}value_{t_{b_{2}}} = 0.946 $, $ p\text{-}value > \alpha \rightarrow $гипотеза $ H_0 $ <b>принимается</b> - параметр <b>незначим</b>.<br>$ t_{b_{3}} = -0.532 $, $ p\text{-}value_{t_{b_{3}}} = 0.608 $, $ p\text{-}value > \alpha \rightarrow $гипотеза $ H_0 $ <b>принимается</b> - параметр <b>незначим</b>.

### Интерпретация параметров

In [158]:
# Судя по качеству модели уровень спроса никак не зависит от цены на товар

## Предпосылки

In [135]:
resid = model.results.resid

In [136]:
dw_test(model)

### Критерий Дарбина-Уотсона для тестирования автокорреляции первого порядка.
<br><br>
$ H_0: \rho_1 = 0, \\ H_1: \rho_1 \neq 0. $
<br><br>
$$ DW = \frac{\sum_{i=2}^{n}{(e_i - e_{i-1})^2}}{\sum_{i=1}^{n}{e_i^2}} \approx 2(1-\rho_1) $$
<br><br>
$ DW = 0.458 $
<br><br>
$ 0.458 < 1.5 \rightarrow $ гипотеза $ H_0 $ <b>отвергается</b> - автокорреляция <b>значительна</b>.

In [137]:
bg_test(model)

### Тест Бройша-Годфри на автокорреляцию.
<br><br>
$ H_0: \rho_1 = \rho_2 = ... = \rho_{\rho} = 0, \\ H_1: \rho_1 \neq \rho_2 \neq ... \neq \rho_{\rho} \neq 0. $
<br><br>
$$ BG = nR^2_{aux} $$
<br><br>


In [138]:
bg_critical, bg_pvalue = sm.stats.acorr_breusch_godfrey(model.results, nlags=1)[:2]

In [161]:
Markdown(fr"""
$ BG = {bg_critical} $<br>
$ p\text{{-}}value = {bg_pvalue} $<br>
<br>
$ {bg_pvalue:.3f} < {0.05} $<br>
$ p\text{{-}}value < \alpha \rightarrow $ гипотеза $ H_0 $ <b>отвергается</b> - автокорреляция <b>присутствует</b>.
""")


$ BG = 8.475674222012627 $<br>
$ p\text{-}value = 0.0035992695845583696 $<br>
<br>
$ 0.004 < 0.05 $<br>
$ p\text{-}value < \alpha \rightarrow $ гипотеза $ H_0 $ <b>отвергается</b> - автокорреляция <b>присутствует</b>.


In [162]:
exog = model.results.model.exog
endog = model.results.model.endog

In [163]:
Markdown(fr"""
### Тест Голдфельда-Квандта
<br><br>
$
H_0: S^2 = S_1^2 = ... = S_n^2 , \\
H_1: S_i^2 = ... = S^2 x_{{ij}}^2 .
$
<br><br>
$ GQ = \frac{{\sum{{e_{{i пред. большая}}^2}}}}{{\sum{{e_{{i пред. меньшая}}^2}}}} $
""")


### Тест Голдфельда-Квандта
<br><br>
$
H_0: S^2 = S_1^2 = ... = S_n^2 , \\
H_1: S_i^2 = ... = S^2 x_{ij}^2 .
$
<br><br>
$ GQ = \frac{\sum{e_{i пред. большая}^2}}{\sum{e_{i пред. меньшая}^2}} $


In [164]:
gq_critical, gq_pvalue = sm.stats.het_goldfeldquandt(endog, exog)[:2]

In [165]:
Markdown(fr"""
$ GQ = {gq_critical} $<br>
$ p\text{{-}}value = {gq_pvalue} $<br>
<br>
$ {gq_pvalue:.3f} < {0.05} $<br>
$ p\text{{-}}value < \alpha \rightarrow $ гипотеза $ H_0 $ <b>отклоняется</b> - гетероскедастичность <b>значима</b>.
""")


$ GQ = 11.197302847656132 $<br>
$ p\text{-}value = 0.04060407967153056 $<br>
<br>
$ 0.041 < 0.05 $<br>
$ p\text{-}value < \alpha \rightarrow $ гипотеза $ H_0 $ <b>отклоняется</b> - гетероскедастичность <b>значима</b>.


In [144]:
Markdown(fr"""
### Тест Бреуша-Пагана
<br><br>
$
H_0: S^2 = S_1^2 = ... = S_n^2 , \\
H_1: S_i^2 = ... = S^2 (x_1, ..., x_k).
$
<br><br>
$ BP = ? $
""")


### Тест Бреуша-Пагана
<br><br>
$
H_0: S^2 = S_1^2 = ... = S_n^2 , \\
H_1: S_i^2 = ... = S^2 (x_1, ..., x_k).
$
<br><br>
$ BP = ? $


In [145]:
bp_critical, bp_pvalue = sm.stats.het_breuschpagan(resid, exog)[2:4]

In [146]:
Markdown(fr"""
$ BP = {bp_critical} $<br>
$ p\text{{-}}value = {bp_pvalue} $<br>
<br>
$ {bp_pvalue:.3f} > {0.05} $<br>
$ p\text{{-}}value > \alpha \rightarrow $ гипотеза $ H_0 $ <b>принимается</b> - гетероскедастичность <b>незначима</b>.
""")


$ BP = 1.654701300269688 $<br>
$ p\text{-}value = 0.2451619990832603 $<br>
<br>
$ 0.245 > 0.05 $<br>
$ p\text{-}value > \alpha \rightarrow $ гипотеза $ H_0 $ <b>принимается</b> - гетероскедастичность <b>незначима</b>.


# Мультипликаторы

### Краткосрочный мультипликатор

In [148]:
model.params['x1']

-0.07816007265481517

### Долгосрочный мультипликатор

In [152]:
model.params[['x1', 'x2', 'x3']].sum()

-0.48387635385306416

### Средний лаг

In [154]:
lag_contrib = model.params[['x1', 'x2', 'x3']] / model.params[['x1', 'x2', 'x3']].sum()
lag_contrib

x1    0.161529
x2   -0.120287
x3    0.958758
dtype: float64

In [157]:
(lag_contrib * range(1, 4)).sum()

2.797228640103969

## Тест на длинную-короткую регрессию

In [96]:
pairs_model = LinearRegression(y_train, X_train[['x', 'const']])

In [119]:
print(pairs_model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                 -0.285
Method:                 Least Squares   F-statistic:                    0.1122
Date:                Sat, 18 Dec 2021   Prob (F-statistic):              0.951
Time:                        16:29:08   Log-Likelihood:                -55.738
No. Observations:                  13   AIC:                             119.5
Df Residuals:                       9   BIC:                             121.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x             -0.0782      0.799     -0.098      0.9

D:\IT\Coding\GitHub Projects\My-University\III Курс\(python) Эконометрика\venv\Lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [99]:
Markdown(fr"""
$
H_0: b_{{k-q+1}} = ... = b_k = 0, //
H_1: b_{{k-q+1}}^2 = ... = b_k^2 > 0.
$
<br><br>
$ F_{{набл}} = \frac{{R_{{UR}}^2 - R_R^2}}{{1 - R_{{UR}}^2}} \cdot \frac{{n - m}}{{q}} $<br>
$ R_{{UR}}^2 =  $ - коэффициент детерминации для длинной модели регрессии;<br>
$ R_R^2 =  $ - коэффициент детерминации для короткой модели регрессии;<br>
$ n $ - кол-во наблюдений;<br>
$ m $ - кол-во параметров длинной модели;<br>
$ q $ - кол-во удаляемых факторов.
""")


$
H_0: b_{k-q+1} = ... = b_k = 0, //
H_1: b_{k-q+1}^2 = ... = b_k^2 > 0.
$
<br><br>
$ F_{набл} = \frac{R_{UR}^2 - R_R^2}{1 - R_{UR}^2} \cdot \frac{n - m}{q} $<br>
$ R_{UR}^2 =  $ - коэффициент детерминации для длинной модели регрессии;<br>
$ R_R^2 =  $ - коэффициент детерминации для короткой модели регрессии;<br>
$ n $ - кол-во наблюдений;<br>
$ m $ - кол-во параметров длинной модели;<br>
$ q $ - кол-во удаляемых факторов.


In [107]:
rsquared_gt = model.results.rsquared
rsquared_lt = pairs_model.results.rsquared
n = model.results.nobs
m = model.results.df_model + 1
q = 2

f_value = ((rsquared_gt - rsquared_lt) / (1 - rsquared_gt)) * ((n - m) / q)
f_value

-1.5548576595490327e-15

In [113]:
f_critical = stats.f.ppf(1 - ALPHA, q, n - m)
f_critical

4.25649472909375

In [115]:
Markdown(fr"""
$ {f_value:.3f} < {f_critical:.3f} $<br>
$ F_{{набл}} < F_{{табл}} \rightarrow $ гипотеза $ H_0 $ принимается -
короткая модель предпочтительнее.
""")


$ -0.000 < 4.256 $<br>
$ F_{набл} < F_{табл} \rightarrow $ гипотеза $ H_0 $ принимается -
короткая модель предпочтительнее.


## Адекватность

In [116]:
y_pred_test_intervals = model.results.get_prediction(X_test).summary_frame(ALPHA)
y_pred_test_intervals

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
15,60.020284,67.936385,-93.662496,213.703064,-100.949302,220.98987


In [117]:
print('Для среднего')
all((y_pred_test_intervals['mean_ci_lower'] <= y_test) & (y_test <= y_pred_test_intervals['mean_ci_upper']))  # noqa

Для среднего


True

In [118]:
print('Для индивидуального')
all((y_pred_test_intervals['obs_ci_lower'] <= y_test) & (y_test <= y_pred_test_intervals['obs_ci_upper']))  # noqa

Для индивидуального


True

In [ ]:
# модель адекватна

# Системы одновременных уравнений

In [175]:
def is_ident(h, d):
    if d + 1 == h:
        print('Уравнение точно идентифицируемо.')
    elif d + 1 < h:
        print('Уравнение неидентифицируемо.')
    else:
        print('Уравнение сверхидентифицируемо')

Задание 1
<br>
$ y_{1t} = a_{12} \dot y_{2t} + b_{11} \dot x_{1t} + v_{1t} $ $ (1) $ <br>
$ y_{2t} = a_{21} \dot y_{1t} + b_{22} \dot x_{2t} + v_{2t} $ $ (2) $


In [186]:
H_y_1 = 2
D_x_1 = 1
is_ident(H_y_1, D_x_1)

Уравнение точно идентифицируемо.


In [187]:
H_y_2 = 2
D_x_2 = 1
is_ident(H_y_2, D_x_2)

Уравнение точно идентифицируемо.


In [188]:
# система идентифицируема

Задание 2
<br>
$ Y_{1t} = \beta_{0} + \beta_{1} Y_{2t} + \beta_{2} X_{t} + e_{t} $ $ (1) $ <br>
$ Y_{2t} = \gamma_{0} + \gamma_{1} Y_{1t} + e_{t} $ $ (2) $ <br>
<br>
$ \hat{Y_{1t}} = 2 + 5 X_{t} $ <br>
$ \hat{Y_{2t}} = 1 + 10 X_{t} $<br>
$ \beta_{1} = 0 $

In [189]:
H_y_1 = 1
D_x_1 = 0
is_ident(H_y_1, D_x_1)

Уравнение точно идентифицируемо.


In [190]:
H_y_1 = 2
D_x_1 = 1
is_ident(H_y_1, D_x_1)

Уравнение точно идентифицируемо.


In [191]:
# система идентифицируема